In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

# raw_dataset = pd.read_csv("/data/fast0/datasets/glathida-3.1.0/data/T.csv")
# raw_dataset = pd.read_csv("~/notebooks/glac/data/T.csv")
raw_dataset = pd.read_csv("~/stuff/coding/glacier/glac/data/T.csv")

# , sep='|', names=['col1']
# , delimiter=", ' "

dataset = raw_dataset.copy()

#examine missing data
dataset.head()

2.6.0


ParserError: Error tokenizing data. C error: Expected 1 fields in line 19, saw 2


In [21]:
#drop everything but lat, long, elevation, thickness

dataset = dataset.drop("GlaThiDa_ID", axis=1)
dataset = dataset.drop("GLACIER_DB", axis=1)
dataset = dataset.drop("POLITICAL_UNIT", axis=1)
dataset = dataset.drop("GLACIER_ID", axis=1)
dataset = dataset.drop("GLACIER_NAME", axis=1)
dataset = dataset.drop("SURVEY_DATE", axis=1)
dataset = dataset.drop("MEAN_THICKNESS_UNCERTAINTY", axis=1)
dataset = dataset.drop("MAXIMUM_THICKNESS", axis=1)
dataset = dataset.drop("MAX_THICKNESS_UNCERTAINTY", axis=1)
dataset = dataset.drop("DATA_FLAG", axis=1)
dataset = dataset.drop("ELEVATION_DATE", axis=1)
dataset = dataset.drop("SPONSORING_AGENCY", axis=1)
dataset = dataset.drop("REMARKS", axis=1)
dataset = dataset.drop("SURVEY_METHOD_DETAILS", axis=1)
dataset = dataset.drop("SURVEY_METHOD", axis=1)
dataset = dataset.drop("NUMBER_OF_SURVEY_POINTS", axis=1)
dataset = dataset.drop("NUMBER_OF_SURVEY_PROFILES", axis=1)
dataset = dataset.drop("TOTAL_LENGTH_OF_SURVEY_PROFILES", axis=1)
dataset = dataset.drop("INTERPOLATION_METHOD", axis=1)
dataset = dataset.drop("INVESTIGATOR", axis=1)
dataset = dataset.drop("REFERENCES", axis=1)

#sum up rows with empty cells
dataset.isna().sum()

#drop the remaining null pieces from elevation
dataset = dataset.dropna()
dataset.isna().sum()

KeyError: "['GlaThiDa_ID'] not found in axis"

In [10]:
dataset

,<!DOCTYPE html>
0,"<html class="""" lang=""en"">"
1,"<head prefix=""og: http://ogp.me/ns#"">"
2,"<meta charset=""utf-8"">"
3,"<link rel=""preload"" href=""/assets/application_..."
4,"<link rel=""preload"" href=""/assets/application-..."
...,...
5891,}
5892,//]]>
5893,</script>
5894,</body>


# 

In [ ]:
#split the dataset and reserve some to test what was trained.
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[["LAT", "AREA", "MEAN_SLOPE", "MEAN_THICKNESS"]], diag_kind='kde')

In [ ]:
train_dataset.describe().transpose()

In [ ]:
#separate out features - what will be trained to predict desired attribute
train_features = train_dataset.copy()
test_features = test_dataset.copy()

#define label - attribute training to be picked
train_labels = train_features.pop("MEAN_THICKNESS")
test_labels = test_features.pop("MEAN_THICKNESS")

train_features.describe().transpose()[['mean', 'std']]

# Establish normalization layers

In [ ]:
#build normalizer layers for each model to be run

normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

area = np.array(train_features["AREA"])
area_normalizer = preprocessing.Normalization(input_shape=[1,], axis=None)
area_normalizer.adapt(area)

lat = np.array(train_features["LAT"])
lat_normalizer = preprocessing.Normalization(input_shape=[1,], axis=None)
lat_normalizer.adapt(lat)

lon = np.array(train_features["LON"])
lon_normalizer = preprocessing.Normalization(input_shape=[1,], axis=None)
lon_normalizer.adapt(lon)

slope = np.array(train_features["MEAN_SLOPE"])
slope_normalizer = preprocessing.Normalization(input_shape=[1,], axis=None)
slope_normalizer.adapt(slope)

# build model function to call

In [ ]:
# #how to condense model into callable function with fillable variables? even possible?
# 88_model = tf.keras.Sequential([
#     88_normalizer,
#     layers.Dense(1)
# ])


# 88_model.compile(
#     optimizer=tf.optimizers.Adam(learning_rate=0.1),
#     loss="mean_absolute_error")

# %%time
# history_88 = 88_model.fit(
#     train_features["*^*"], train_labels,
#     epochs=400,
#     # suppress logging
#     verbose=0,
#     # Calculate validation results on 20% of the training data
#     validation_split = 0.2)

# test_results = {}
# test_results["*_model"] = area_model.evaluate(
#     test_features["*^*"],
#     test_labels, verbose=0)

# test_results

# def plot_area(x, y):
#     plt.scatter(train_features["*^*"], train_labels, label='Data')
#     plt.plot(x, y, color='k', label='Predictions')
#     plt.xlabel('Area (km**2)')
#     plt.ylabel('Thickness')
#     plt.xlim((0,20))
#     plt.legend()
    
# x = tf.linspace(0.0, 20, 101)
# y = area_model.predict(x)
# plot_area(x,y)

# def plot_loss(history_area):
#   plt.plot(history_area.history['loss'], label='loss')
#   plt.plot(history_area.history['val_loss'], label='val_loss')
# #   plt.ylim([0, 10])
#   plt.xlabel('Epoch')
#   plt.ylabel('Error [MPG]')
#   plt.legend()
#   plt.grid(True)
# plot_loss(history_area)



# Linear regression with area

In [ ]:
#build elevation model with Sequential
area_model = tf.keras.Sequential([
    area_normalizer,
    layers.Dense(1)
])

area_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.6),
    loss="mean_absolute_error")

history_area = area_model.fit(
    train_features["AREA"], train_labels,
    epochs=1000,
    verbose=0,
    validation_split = 0.2)

test_results = {}
test_results['area_model'] = area_model.evaluate(
    test_features['AREA'],
    test_labels, verbose=0)

test_results

In [ ]:
def plot_area(x, y):
    plt.scatter(train_features['AREA'], train_labels, label='Data')
    plt.plot(x, y, color='k', label='Predictions')
    plt.xlabel('Area (km**2)')
    plt.ylabel('Thickness')
    plt.xlim((0,20))
    plt.legend()
    
x = tf.linspace(0.0, 20, 101)
y = area_model.predict(x)
plot_area(x,y)

In [ ]:
def plot_loss(history_area):
    plt.subplots(figsize=(10,5))
    plt.plot(history_area.history['loss'], label='loss')
    plt.plot(history_area.history['val_loss'], label='val_loss')
    #   plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [Area, km**2]')
    plt.legend()
    plt.grid(True)



In [ ]:
plot_loss(history_area)

# Linear regression with latitude

In [ ]:
#build elevation model with Sequential
lat_model = tf.keras.Sequential([
    lat_normalizer,
    layers.Dense(1)
])

lat_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss="mean_absolute_error")

history_lat = lat_model.fit(
    train_features["LAT"], train_labels,
    epochs=400,
    verbose=0,
    validation_split = 0.2)

test_results["lat_model"] = area_model.evaluate(
    test_features["LAT"],
    test_labels, verbose=0)

test_results

In [ ]:
def plot_latitude(x, y):
    plt.scatter(train_features["LAT"], train_labels, label='Data')
    plt.plot(x, y, color='k', label='Predictions')
    plt.xlabel("Latitude")
    plt.ylabel('Thickness')
    plt.xlim((-90,90))
    plt.legend()
    
x = tf.linspace(0.0, 20, 101)
y = area_model.predict(x)
plot_latitude(x,y)

In [ ]:
plot_loss(history_lat)

# Linear regression with longitude

In [ ]:
lon_model = tf.keras.Sequential([
    lon_normalizer,
    layers.Dense(1)
])

lon_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss="mean_absolute_error")

history_lon = lon_model.fit(
    train_features["LON"], train_labels,
    epochs=400,
    verbose=0,
    validation_split = 0.2)

test_results["lon_model"] = area_model.evaluate(
    test_features["LON"],
    test_labels, verbose=0)

test_results

In [ ]:
def plot_longitude(x, y):
    plt.scatter(train_features["LON"], train_labels, label='Data')
    plt.plot(x, y, color='k', label='Predictions')
    plt.xlabel('LON')
    plt.ylabel('Thickness')
    plt.xlim((0,180))
    plt.legend()
    
x = tf.linspace(0.0, 20, 101)
y = area_model.predict(x)
plot_longitude(x,y)

In [ ]:
plot_loss(history_lon)

# Multiple inputs

In [ ]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
multi_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
multi_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    loss='mean_absolute_error')

In [ ]:
%%time
history_multi = multi_model.fit(
    train_features, train_labels, 
    epochs=1000,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

In [ ]:
#not sure what exactly is going on here but I'm sure this is what needs tweaking
test_results["multi_model"] = multi_model.evaluate(
    test_features,
    test_labels, verbose=0)

test_results

In [ ]:
plot_loss(history_multi)

# DNN regression

### Single variable

In [ ]:
def build_and_compile_model(norm):
    model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01))
    return model

In [ ]:
dnn_area_model = build_and_compile_model(area_normalizer)

In [ ]:
dnn_area_model.summary()

In [ ]:
%%time
history_dnn = dnn_area_model.fit(
    train_features["AREA"], train_labels,
    validation_split=0.2,
    #verbose=0,
    epochs=1500)

In [ ]:
dnn_test_results = {}
dnn_test_results['dnn_area_model'] = dnn_area_model.evaluate(
    test_features["AREA"], test_labels,
    verbose=0)

# Full model (not-quite-working)

In [ ]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
dnn_full_model = build_and_compile_model(normalizer)
dnn_full_model.summary()

In [ ]:
%%time
history_full = dnn_full_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
plot_loss(history_full)

In [ ]:
y = dnn_full_model.predict(test_features)
plt.plot(test_labels,y,'o')
plt.plot((0,150),(0,150),'-')
plt.xlabel('True Area (km**2)')
plt.ylabel('Model Area (km**2)')

In [ ]:
plt.subplots()
x = np.reshape(test_labels.values, (-1, 1)) # Training values in a n-by-1 array
pct_error = (x-y)/x
plt.hist(pct_error.flatten())
plt.xlabel('Percent Error')
plt.ylabel('Count')

In [ ]:
test_results["dnn_full_model"] = dnn_full_model.evaluate(
    test_features,
    test_labels, verbose=0)

test_results